# Monte Carlo Simulation for FE
## IEOR 4703

### Implementation of a delta-hedging strategy for a call option

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from scipy.stats import norm

In [ ]:

def BMS_d1(S, K, r, q, sigma, tau):
    ''' Computes d1 for the Black Merton Scholes formula '''
    d1 = 1.0*(np.log(1.0 * S/K) + (r - q + sigma**2/2) * tau) / (sigma * np.sqrt(tau))
    return d1

def BMS_d2(S, K, r, q, sigma, tau):
    ''' Computes d2 for the Black Merton Scholes formula '''
    d2 = 1.0*(np.log(1.0 * S/K) + (r - q - sigma**2/2) * tau) / (sigma * np.sqrt(tau))
    return d2

def BMS_price(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the Black Merton Scholes price for a 'call' or 'put' option '''
    tau = T - t
    d1 = BMS_d1(S, K, r, q, sigma, tau)
    d2 = BMS_d2(S, K, r, q, sigma, tau)
    if type_option == 'call':
        price = S * np.exp(-q * tau) * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    elif type_option == 'put':
        price = K * np.exp(-r * tau) * norm.cdf(-d2) - S * np.exp(-q * tau) * norm.cdf(-d1) 
    return price

def BMS_delta(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the delta for a call or a put. '''
    tau = T - t
    d1 = BMS_d1(S, K, r, q, sigma, tau)
    if type_option == 'call':
        delta = np.exp(-q * tau) * norm.cdf(d1)
    elif type_option == 'put':
        delta = np.exp(-q * tau) * (norm.cdf(d1) - 1)
    return delta

In [ ]:
spot = 100
K = 110
r = 0.15
q = 0.15
sig = 0.22

maturity = 1

m = 365
dt = maturity/m

n_sim = 100

Compute the true value of the call

In [ ]:
C = BMS_price('call', spot, K, r, q, sig, maturity)
print('C = ' + str(C))

C = 4.340225998353667


### Q3

Delta-hedging strategy -- with r>0 and q>0

In [ ]:
np.random.seed(4123456)


In [ ]:
synthetic_C = np.zeros(n_sim)

st = time()

for j in range(n_sim):
    
    # reset for each path
    S = spot
    T = maturity
    delta_C_prev = 0.0
    
    for i in range(m):
        
        delta_C = BMS_delta('call', S, K, r, q, sig, T)
        
        synthetic_C[j] += (delta_C - delta_C_prev)*S
        # Note - net position is always delta_c
        if i > 0:
          synthetic_C[j] -= delta_C_prev*S_prev*(np.exp(q*dt) - 1) /np.exp(i*r*dt)
    
        delta_C_prev = delta_C

        S_prev = S
        z = np.random.randn()
        S = S * np.exp((r - q - sig*sig/2)*dt + sig*np.sqrt(dt)*z)
        
        T = T - dt
    
    synthetic_C[j] += (-delta_C*S + np.maximum(S-K,0))*np.exp(-m*r*dt)


In [ ]:
print('C = ' + str(C))
print('mean_synthetic_C = ' + str(np.mean(synthetic_C)))
print('std_synthetic_C = ' + str(np.std(synthetic_C, ddof=1)))

C = 4.340225998353667
mean_synthetic_C = 4.980484867743424
std_synthetic_C = 5.0412164319856485


### Q4

put option with r>0 and q>0

In [ ]:
synthetic_P = np.zeros(n_sim)

st = time()

for j in range(n_sim):
    
    # reset for each path
    S = spot
    T = maturity
    delta_P_prev = 0.0
    
    for i in range(m):
        
        delta_P = BMS_delta('put', S, K, r, q, sig, T)
        
        synthetic_P[j] += (delta_P - delta_P_prev)*S
        # Note - net position is always delta_c
        if i > 0:
          synthetic_P[j] -= delta_P_prev*S_prev*(np.exp(q*dt) - 1) /np.exp(i*r*dt)
    
        delta_P_prev = delta_P

        S_prev = S
        z = np.random.randn()
        S = S * np.exp((r - q - sig*sig/2)*dt + sig*np.sqrt(dt)*z)
        
        T = T - dt
    
    synthetic_P[j] += (-delta_P*S + np.maximum(K-S,0))*np.exp(-m*r*dt)


In [ ]:
P = BMS_price('put', spot, K, r, q, sig, maturity)
print('P = ' + str(P))
print('mean_synthetic_P = ' + str(np.mean(synthetic_P)))
print('std_synthetic_P = ' + str(np.std(synthetic_P, ddof=1)))

P = 12.94730576260423
mean_synthetic_P = 12.386791872001877
std_synthetic_P = 4.699566726631207


### Q5

BMS Model with non-constant volatility (Call)

In [ ]:

synthetic_C = np.zeros(n_sim)

st = time()

for j in range(n_sim):
    
    # reset for each path
    S = spot
    T = maturity
    delta_C_prev = 0.0
    sig_prev = sig

    for i in range(m):
        sig = sig_prev
        delta_C = BMS_delta('call', S, K, r, q, sig, T)
        
        synthetic_C[j] += (delta_C - delta_C_prev)*S
        # Note - net position is always delta_c
        if i > 0:
          synthetic_C[j] -= delta_C_prev*S_prev*(np.exp(q*dt) - 1) /np.exp(i*r*dt)
    
        delta_C_prev = delta_C

        S_prev = S
        z = np.random.randn()
        S = S * np.exp((r - q - sig*sig/2)*dt + sig*np.sqrt(dt)*z)
        sig = sig_prev - 0.001*(S - S_prev)
        T = T - dt
    
    synthetic_C[j] += (-delta_C*S + np.maximum(S-K,0))*np.exp(-m*r*dt)


In [ ]:
print('mean_synthetic_C = ' + str(np.mean(synthetic_C)))
print('std_synthetic_C = ' + str(np.std(synthetic_C, ddof=1)))

mean_synthetic_C = 4.782915151952462
std_synthetic_C = 4.94105928763795
